This Jupyter notebook is used for:
- Convert .mrd to a an h5 file.
- Convert the ismrmrd object into a numpy matrix.
- Add the headers to the h5 file
- Remove the zero-padding from the UMCG data if needed
- Remove the zero-padding from the RUMC (Radboud) data if needed
- Transform the UMCG data (phase encoding) into the same dimensions as the NYU data
- Visualize the UMCG averages. How are they structured and related to each other
- Perform RSS reconstruction on UMCG and NYU data.

Later: (todo)
- It will later also add the segmentations to the h5 file.
- Add the converted calibration data to the h5 file.
- Add the DICOM images to the h5 file.



# Constants and Imports

In [1]:
import os
from pathlib import Path
from tqdm import tqdm
import pydicom
import SimpleITK as sitk
import datetime

from helper_functions import *

DEBUG      = True
KEY        = 'x'

# Windows path external ssd: E:\quintin\umcg_ksp_ws
WORKDIR = Path("E:/quintin/umcg_ksp_ws")        # Windows, Quintin, external ssd

TEMPDIR    = Path(WORKDIR, 'tmp')
H5DIR      = Path(WORKDIR, 'output', 'h5s')

your_path= Path("/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace")

# # H5 files
FPATH_UMCG_H5_SAMPLE = Path(WORKDIR, "output", "h5s", "ANON2784451_pst_T2.h5")
FPATH_NYU_H5_SAMPLE  = Path(your_path, "prostate_nyu/training_T2_1/file_prostate_AXT2_0001.h5")

# MRD files
FPATH_UMCG_MRD_SAMPLE_CALIB = Path(your_path, '/output/anon_kspaces/0001_patient_umcg_done/meas_MID00202_FID688156_T2_TSE_tra_obl-out_1.mrd')
FPATH_UMCG_MRD_SAMPLE       = Path(your_path, '/output/anon_kspaces/0001_patient_umcg_done/meas_MID00202_FID688156_T2_TSE_tra_obl-out_2.mrd')

print(WORKDIR)
print(TEMPDIR)
print(H5DIR)
print(os.getcwd())

ic| WORKDIR: WindowsPath('E:/quintin/umcg_ksp_ws')
ic| TEMPDIR: WindowsPath('E:/quintin/umcg_ksp_ws/tmp')
ic| H5DIR: WindowsPath('E:/quintin/umcg_ksp_ws/output/h5s')
ic| os.getcwd(): 'c:\\Users\\LohuizenQY\\local_docs\\repos\\umcglib\\src\\umcglib\\kspace'


'c:\\Users\\LohuizenQY\\local_docs\\repos\\umcglib\\src\\umcglib\\kspace'

# Create H5s from MRD files (UMCG)

In [ ]:
def reorder_interleaved_slices_to_sequential(npdata):
    '''
    Reorder the kspace data from interleaved to all even slices first and then all odd slices
    Arguments:
        - npdata: numpy array of kspace data in shape (navgs, nslices, ncoils, rNx, eNy + 1) complex
    Returns:
        - data: numpy array of kspace data in shape (navgs, nslices, ncoils, rNx, eNy + 1) complex
    '''

    assert npdata.ndim == 5, "Input data must be 5D, with dimensions [navgs, nslices, ncoils, rNx, eNy + 1]"

    # Get the number of slices and calculate the half value (rounded up)
    nslices = npdata.shape[1]
    mid_slice = (nslices // 2) + 1

    # Create output array of the same shape and data type as the input array 
    reordered_data = np.zeros_like(npdata, dtype=np.complex64)

    # Even slices first (0, 2, 4, ...) then odd slices (1, 3, 5, ...)
    reordered_data[:, ::2] = npdata[:, :mid_slice]
    reordered_data[:, 1::2] = npdata[:, mid_slice:]

    safe_rss_to_nifti_file(reordered_data)
    input("Press Enter to continue...")

    return reordered_data


def build_kspace_array_from_mrd_umcg(fpath: str, verbose=True):
    '''
    Arguments:
        - fpath: path to the .mrd file
    
    Returns:
        - kspace: numpy array of kspace data in shape (navgs, nslices, ncoils, rNx, eNy + 1) complex
    '''
    if verbose:
        print(f"Building kspace from .mrd file")

    # Check if the kspace already exists in the TEMPDIR
    if os.path.exists(os.path.join(TEMPDIR, "kspace_0003.npy")):
        print(f"kspace already exists at {os.path.join(TEMPDIR, 'kspace_0003.npy')}")
        kspace = np.load(os.path.join(TEMPDIR, "kspace_0003.npy"))
        return kspace
    else:

        # Read the header and get encoding information
        dset = ismrmrd.Dataset(fpath, create_if_needed=False)
        header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
        enc = header.encoding[0]

        # Determine some parameters of the acquisition
        ncoils     = header.acquisitionSystemInformation.receiverChannels
        nslices    = enc.encodingLimits.slice.maximum + 1 if enc.encodingLimits.slice is not None else 1
        eNy        = enc.encodedSpace.matrixSize.y
        rNx        = enc.reconSpace.matrixSize.x
        # eTL        = 25 if DEBUG else echo_train_length(dset)
        # eTC        = 11 if DEBUG else echo_train_count(dset, echo_train_len=eTL)
        firstacq   = 1 #if DEBUG else get_first_acquisition(dset)
        navgs      = 3 #if DEBUG else get_num_averages(firstacq=firstacq, dset=dset)

        # Initialize the kspace data array
        kspace = np.zeros((navgs, nslices, ncoils, rNx, eNy + 1), dtype=np.complex64)
        print(f"Filling kspace array from mrd object to shape {kspace.shape}...\n Num Acq: {dset.number_of_acquisitions()}")

        # Loop through the rest of the acquisitions and fill the data array with the kspace data
        for acqnum in range(firstacq, dset.number_of_acquisitions()):
            
            if acqnum % 1000 == 0:
                print(f"{acqnum/dset.number_of_acquisitions() * 100:.1f}%", end=" ", flush=True)
            
            acq    = dset.read_acquisition(acqnum)
            slice1 = acq.idx.slice
            y      = acq.idx.kspace_encode_step_1
            avg    = acq._head.idx.average

            # Each acquisition is a 2D array of shape (coil, rNx) complex
            kspace[avg, slice1, :, :, y] = acq.data
    
    # store the kspace as npy file to the TEMPDIR if it does not exist yet
    if not os.path.exists(os.path.join(TEMPDIR, "kspace_0003.npy")):
        np.save(os.path.join(TEMPDIR, "kspace_0003.npy"), kspace)
        print(f"kspace saved to {os.path.join(TEMPDIR, 'kspace_0003.npy')}")

    # Reorder the data from even and odd number of slices being interleaved to all even slices first and then all odd slices

    pat_dcm_dir = "/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/input/sectra_export_pat0001_pat0010_anon_dicoms/ANON5046358/2022-05-10/T2W_TRA"
    # method 3 - Reorder based on the dicom slice numbers and their respective acquisition times
    # remap_indices = get_slice_remap_instanceNumber_vs_imagePatientPosition(pat_dcm_dir, verbose=True)
    # remap_indices = get_slice_remap_instanceNumber_vs_acquisitionTime(pat_dcm_dir, verbose=True)
    # remap_indices = get_slice_remap_by_image_position(pat_dcm_dir, verbose=True)
    # remap_indices = get_slice_remap_by_instace_number(pat_dcm_dir, verbose=True)
    remap_indices = get_slice_remap_based_on_acquisition_time(pat_dcm_dir, verbose=True)

    input(f"in build_kspace_array_from_mrd_umcg v1:     enter to continue..")
    kspace = kspace[:, remap_indices, :, :, :]

    safe_rss_to_nifti_file(kspace, fname_part="_reorder_from_dcms_instNum_vs_Patpos_", do_round=True)
    input(f"in build_kspace_array_from_mrd_umcg v2:     enter to continue..")


    return reorder_interleaved_slices_to_sequential(kspace)


def process_mrd_to_h5(
        fpath: str,
        fpath_hf: str,
        max_mag_ref: float,
        do_rm_zero_pad: bool,
        do_norm_to_ref: bool,
        pat_seq_id: str,
        pat_anon_id: str,
        phase_crop_shape: tuple = None,
    ) -> None:
    '''
    Description:
        This function creates an h5 file from a .mrd file. The h5 file contains the kspace and the ismrmrd header.
        The kspace is cropped in the phase direction to the shape of the NYU dataset.
        The kspace is normalized to the reference magnitude of the NYU dataset.
        The h5 file is named with the anonymized patient id.
    Args:
        fpath (str): The path to the .mrd file.
        fpath_hf (str): The path to the h5 file.
        phase_crop_shape (tuple): The shape to crop the kspace to.
        max_mag_ref (float): The reference magnitude.
        do_rm_zero_pad (bool): If True, the zero padding is removed.
        do_norm_to_ref (bool): If True, the magnitude is normalized to the reference magnitude.
    '''
    
    max_for_now  = 0.0004   # change in the future to something that makes sense
    norm_for_now = 0.12    # change in the future to something that makes sense
    
    # Construct the kspace array from the sequentail MRD object.
    kspace = build_kspace_array_from_mrd_umcg(fpath)

    if do_rm_zero_pad:
        kspace = remove_zero_padding(kspace)

    # Crop the kspace in the phase dir and obtain the transformed headers. Simply extracts the headers as is, if the crop shape is equal to the kspace shape.
    kspace, trans_hdrs = crop_kspace_in_phase_direction(kspace, target_shape=phase_crop_shape, fpath_mrd=fpath)

    if do_norm_to_ref:
        kspace = normalize_to_reference(kspace, max_magni_ref=max_mag_ref)

    with h5py.File(fpath_hf, 'r+') as hf:
        if not has_kspace_key(fpath_hf):
            hf.create_dataset('ismrmrd_header', data=trans_hdrs)
            hf.create_dataset('kspace', data=kspace)

        if not has_correct_shape(fpath_hf):
            raise Exception(f"kspace shape is not correct. Shape: {hf['kspace'].shape}")

        if len(dict(hf.attrs)) == 0:
            hf.attrs['acquisition'] = 'AXT2'
            hf.attrs['max'] = max_for_now
            hf.attrs['norm'] = norm_for_now
            hf.attrs['patient_id'] = pat_anon_id
            hf.attrs['patient_id_seq'] = pat_seq_id


def create_t2w_h5_from_mrd_umcg(
        fpath: str,
        workdir: str,
        key: str,
        do_rm_zero_pad   = True,
        phase_crop_shape = None,  # This is the NYU shape.
        do_norm_to_ref   = True,
        max_mag_ref      = 0.006096669,  # Based on one patient of the NYU dataset the max_magnitude reference: # MUST IN THE FUTURE BE DETERMINED ON THE ENTIRE NYU dataset. Or return the NYU RIM model on normalized data.
) -> None:
    '''
    Description:
        This function creates an h5 file from a .mrd file. The h5 file contains the kspace and the ismrmrd header.
        The kspace is cropped in the phase direction to the shape of the NYU dataset.
        The kspace is normalized to the reference magnitude of the NYU dataset.
        The h5 file is named with the anonymized patient id.
    Args:
        workdir (str): The path to the workdir.
        key (str): The key to the mapping dictionary.
        do_rm_zero_pad (bool): If True, the zero padding is removed.
        phase_crop_shape (tuple): The shape to crop the kspace to.
        do_norm_to_ref (bool): If True, the magnitude is normalized to the reference magnitude.
        max_mag_ref (float): The reference magnitude.
    '''
    print(f"!!!!!!max_mag_ref is set to {max_mag_ref}. This must be changed later to the max of the NYU dataset.")
    print(f"!!!!!!norm and max of the h5 file must be calculated later and set properly.")

    pat_seq_id  = get_patient_id(fpath).strip()
    pat_anon_id = get_anon_pat_id_from_pat_id(pat_seq_id, get_mapping_patient_ids(workdir, key))
    print(f"Processing patient {pat_seq_id} with pat_anon_id {pat_anon_id}")

    fpath_hf = os.path.join(workdir, "output", "h5s", f"{pat_anon_id}_pst_T2.h5")

    if not os.path.exists(fpath_hf):
        with h5py.File(fpath_hf, 'w') as hf:
            print(f"created h5 file at {fpath_hf}")
    else:
        print(f"\tH5 file already exists: {fpath_hf}")

    process_mrd_to_h5(
        fpath            = fpath,
        fpath_hf         = fpath_hf,
        phase_crop_shape = phase_crop_shape,
        max_mag_ref      = max_mag_ref,
        do_rm_zero_pad   = do_rm_zero_pad,
        do_norm_to_ref   = do_norm_to_ref,
        pat_seq_id       = pat_seq_id,
        pat_anon_id      = pat_anon_id
    )


# Get the paths to the t2 tra files
t2_tra_paths = get_t2_tra_paths(WORKDIR, verbose=False)
print(t2_tra_paths)

t2_tra_paths = [t2_tra_paths[12]]   # patient 0003

# loop over the paths and create the h5 files
for f_idx, fpath in enumerate(t2_tra_paths):
    create_t2w_h5_from_mrd_umcg(
        fpath            = fpath,
        workdir          = WORKDIR,
        key              = KEY,
        do_rm_zero_pad   = True, 
        phase_crop_shape = None, # (3, 30, 20, 640, 551) # same shape. We will do the phase cropping in the RIM code. Make a new dataset there with a transform that does this operation. At a later stage in this code it will also include kspace zero-padding.
        do_norm_to_ref   = True,
        max_mag_ref      = 0.006096669,  # Based on one patient of the NYU dataset the max_magnitude reference: # MUST IN THE FUTURE BE DETERMINED ON THE ENTIRE NYU dataset. Or return the NYU RIM model on normalized data.
    )

    if DEBUG:
        print("debug mode enabled. breaking after 1 patient dir..")
        break

# load the kspace from npy file

In [2]:
# load the numpy file if it exists
if os.path.exists(os.path.join(TEMPDIR, "kspace_0003.npy")):
    kspace = np.load(os.path.join(TEMPDIR, "kspace_0003.npy"))
    print(f"kspace loaded from {os.path.join(TEMPDIR, 'kspace_0003.npy')}")

kspace loaded from /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/tmp/kspace_0003.npy


In [3]:
# add average 1 and 2 together but keep the dimension
kspace = np.sum(kspace, axis=0, keepdims=True)

# add coil 1 and 2 together but keep the dimension
kspace = np.sum(kspace, axis=2, keepdims=True)

In [4]:
kspace.shape

(1, 30, 1, 768, 814)

# load in the kspace as is and try to reorder it here

In [16]:
def safe_rss_to_nifti_file(kspace: np.ndarray, fname_part, do_round=True) -> None:
    """
    Description:
        Perform root sum of squares reconstruction on the given k-space.
    Args:
        kspace (np.ndarray): The k-space data. 5D array with dimensions [num_averages, num_slices, num_coils, num_readout_points, num_phase_encode_steps]
    Returns:
        None
    """

    assert kspace.ndim == 5, "image should have 5 dimensions: (n_avg, n_slices, n_coils, n_freq, n_phase)"

    rss_recon = rss_recon_from_ksp(kspace, averages=(0,1), verbose=True, printkey="safe_rss_to_nifti_file_reorder")

    if do_round:
        rss_recon = np.round(rss_recon*1000, decimals=3)

    save_numpy_rss_as_nifti(rss_recon, fname=f"{fname_part}rss_recon", dir=TEMPDIR)


def get_slice_remap_based_instanceID_and_patPosition(dicom_dir: str, verbose=False):

    instance_numbers = []
    acquisition_times = []
    for filename in os.listdir(dicom_dir):
        ds = pydicom.dcmread(os.path.join(dicom_dir, filename))

        instance_numbers.append(ds.InstanceNumber)
        acquisition_times.append(ds.ImagePositionPatient[-1])

    # sort both the instance numbers and the acquisition times based on acquistion times small first then large
    sorted_instance_numbers = [x for _,x in sorted(zip(acquisition_times, instance_numbers))]
    # nicely print the sorted lists again:

    for i in range(len(sorted_instance_numbers)):
        print(f"{sorted_instance_numbers[i]} : {acquisition_times[i]}")

    return np.array(sorted_instance_numbers) - 1  # -1 to make it zero-based for Python array indexing


def reorder_interleaved_slices_to_sequential(kspace_array: np.ndarray) -> np.ndarray:
    """
    Reorder k-space slices from interleaved to sequential: all even slices first, followed by all odd slices.
    
    Parameters:
        kspace_array: Numpy array of shape [num_averages, num_slices, num_coils, num_readout_points, num_phase_encode_steps].
        
    Returns:
        reordered_k_space_data: Numpy array with reordered slices, maintaining the same shape as the input.
    """
    
    # Validate that the input data has the expected 5D shape
    assert kspace_array.ndim == 5, "Input data must be a 5D array with dimensions [num_averages, num_slices, num_coils, num_readout_points, num_phase_encode_steps]"
    
    # Log the reordering operation for debugging purposes
    print("\nReordering k-space slices...")
    
    # Compute the mid-point index for reordering slices, rounding up for odd numbers
    num_slices = kspace_array.shape[1]
    mid_slice_idx = ((num_slices // 2) + 1) if num_slices % 2 == 1 else num_slices // 2
    
    # Initialize an output array with the same shape and data type as the input array
    reordered_k_space_data = np.zeros_like(kspace_array, dtype=np.complex64)
    
    reordered_k_space_data[:, 1::2] = kspace_array[:, :mid_slice_idx]
    reordered_k_space_data[:, ::2] = kspace_array[:, mid_slice_idx:]
    
    safe_rss_to_nifti_file(reordered_k_space_data, fname_part="_reorder_ori_tiago")

    return reordered_k_space_data


def reorder_k_space_even_odd(k_space: np.ndarray) -> np.ndarray:
    """
    Rearranges the k-space data by interleaving even and odd indexed slices.

    Parameters:
    - k_space (numpy.ndarray): The input k-space data to be rearranged.

    Returns:
    - interleaved_k_space (numpy.ndarray): The k-space data with interleaved slices.
    """
    
    # Initialize a new complex array to store the interleaved k-space data
    interleaved_k_space = np.zeros_like(k_space, dtype=np.complex64)

    # Calculate the middle index for slicing the array into two halves
    num_slices = k_space.shape[1]
    middle_index = (num_slices + 1) // 2  # Handles both odd and even cases

    # Interleave even and odd indexed slices
    interleaved_k_space[:, ::2] = k_space[:, middle_index:]  # Place the second half at even indices
    interleaved_k_space[:, 1::2] = k_space[:, :middle_index]  # Place the first half at odd indices

    return interleaved_k_space


def reorder_kspace_inplace(ksp_array: np.ndarray) -> np.ndarray:
    """
    Rearrange k-space slices in place based on interleaved order.

    Parameters:
    - ksp_array (numpy.ndarray): 5D array of shape [num_averages, num_slices, num_coils, num_readout_points, num_phase_encode_steps].

    Returns:
    - ksp_array (numpy.ndarray): Rearranged 5D array.
    """

    # Validate that the input data has the expected 5D shape
    assert ksp_array.ndim == 5 or ksp_array.ndim==3, "Input data must be a 3D, or 5D array"

    num_slices = ksp_array.shape[1]
    half_idx = (num_slices + 1) // 2

    # Compute indices for the reordered array
    even_indices = np.arange(half_idx) * 2
    odd_indices = even_indices + 1
    interleaved_indices = np.zeros(num_slices, dtype=np.int32)

    interleaved_indices[:half_idx] = even_indices
    interleaved_indices[half_idx:num_slices] = odd_indices[:num_slices - half_idx]

    # Rearrange the array based on computed indices
    ksp_array[:, :] = ksp_array[:, interleaved_indices]

    return ksp_array


###############################
print(f" ksapce shape: {kspace.shape}")

pat_dcm_dir = "/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/input/sectra_export_pat0001_pat0010_anon_dicoms/ANON5046358/2022-05-10/T2W_TRA"
# remap_indices = get_slice_remap_instanceNumber_vs_imagePatientPosition(pat_dcm_dir, verbose=True)
# remap_indices = get_slice_remap_instanceNumber_vs_acquisitionTime(pat_dcm_dir, verbose=True)
# remap_indices = get_slice_remap_by_image_position(pat_dcm_dir, verbose=True)
# remap_indices = get_slice_remap_by_instace_number(pat_dcm_dir, verbose=True)

if True:
    if False:   
        # method 1 - Acquisition time
        remap_indices = get_slice_remap_based_instanceID_and_patPosition(pat_dcm_dir, verbose=True)
        kspace_reordered = kspace[:, remap_indices, :, :, :]

    if True:
        # method 2 - Simple zipping
        kspace_reordered = reorder_k_space_even_odd(kspace)

if False:
    kspace_reordered = reorder_kspace_inplace(kspace)

# create a timestamp with current date and time
now = datetime.datetime.now()
timestring = now.strftime("%d-%H-%M")

# save image space
safe_rss_to_nifti_file(kspace_reordered, fname_part=f"_roerdered_ksp_{timestring}_", do_round=True)
safe_rss_to_nifti_file(kspace, fname_part=f"_not_reordered_{timestring}_", do_round=True)

 ksapce shape: (1, 30, 1, 768, 814)
Number of slices: 30
Half index: 15
safe_rss_to_nifti_file_reorder - kspace shape: ((1, 30, 1, 768, 814))
safe_rss_to_nifti_file_reorder - Image  shape: (30, 768, 814)
safe_rss_to_nifti_file_reorder - kspace shape collapsed from averages: (30, 1, 768, 814)
Saved image to /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/tmp/_roerdered_ksp_25-14-42_rss_recon.nii.gz
safe_rss_to_nifti_file_reorder - kspace shape: ((1, 30, 1, 768, 814))
safe_rss_to_nifti_file_reorder - Image  shape: (30, 768, 814)
safe_rss_to_nifti_file_reorder - kspace shape collapsed from averages: (30, 1, 768, 814)
Saved image to /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_ksp_umcg/workspace/tmp/_not_reordered_25-14-42_rss_recon.nii.gz


## Create H5s from MRD files (RUMC)

In [ ]:
def get_headers_from_ismrmrd_rumc(fpath: str, verbose=False) -> dict:
    '''
    Description:
        - get the headers from a .mrd file
    Arguments:
        - fpath: path to the .mrd file
    Returns:
        - headers: dictionary with the headers
    '''
    dset = ismrmrd.Dataset(fpath, 'dataset', create_if_needed=False)
    # what is the type of dset
    print(type(dset))
    # what is within: ismrmrd.hdf5.Dataset
    print(dir(dset))

    print("doneeee")
    headers = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())

    if verbose:
        print("\tHEADERS found in .mrd file")
        print(f"\t\tHEADERS.SUBJECTINFORMATION")
        print_headers(headers.subjectInformation)
        print(f"\t\tHEADERS.MEASUREMENTINFORMATION")
        print_headers(headers.measurementInformation)
        print(f"\t\tHEADERS.ACQUISITIONSYSTEMINFORMATION")
        print_headers(headers.acquisitionSystemInformation)
        print(f"\t\tHEADERS.SEQUENCEPARAMETERS")
        print_headers(headers.sequenceParameters)
        print(f"\t\tHEADERS.USERPARAMETERS")
        print_headers(headers.userParameters)
        print(f"\t\tHEADERS.EXPERIMENTALCONDITIONS")
        print_headers(headers.experimentalConditions)
        print(f"\t\tHEADERS.ENCODEDSPACE")
        print_headers(headers.encoding[0].encodedSpace)
        print(f"\t\tHEADERS.TRAJECTORY")
        print_headers(headers.encoding[0].trajectory)
        print(f"\t\tHEADERS.RECONSPACE")
        print_headers(headers.encoding[0].reconSpace)
        print(f"\t\tHEADERS.ENCODINGLIMITS")
        print_headers(headers.encoding[0].encodingLimits)
        print(f"\t\tHEADERS.PARALLELIMAGING")
        print_headers(headers.encoding[0].parallelImaging)
        print(type(headers))

    return headers


def build_np_from_ismrmrd_rumc(fpath: str, DEBUG=False, verbose=True):
    '''
    Arguments:
        - fpath: path to the .mrd file
    
    Returns:
        - kspace: numpy array of kspace data in shape (navgs, nslices, ncoils, rNx, eNy + 1) complex
    '''
    if verbose:
        print(f"Building kspace from .mrd file")

    # lets time this whole function and print it
    start = time.time()

    # Read the header and get encoding information
    dset = ismrmrd.Dataset(fpath, dataset_name='dataset_2', create_if_needed=False)
    header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
    enc = header.encoding[0]

    # Determine some parameters of the acquisition
    ncoils     = header.acquisitionSystemInformation.receiverChannels
    nslices    = enc.encodingLimits.slice.maximum + 1 if enc.encodingLimits.slice is not None else 1
    eNy        = enc.encodedSpace.matrixSize.y
    rNx        = enc.reconSpace.matrixSize.x
    eTL        = 25 if DEBUG else echo_train_length(dset)
    eTC        = 11 if DEBUG else echo_train_count(dset, echo_train_len=eTL)
    firstacq   = 1 if DEBUG else get_first_acquisition(dset)
    navgs      = 3 if DEBUG else get_num_averages(firstacq=firstacq, dset=dset)

    if verbose:
        print(f"ncoils: {ncoils}")
        print(f"nslices: {nslices}")
        print(f"eNy: {eNy}")
        print(f"rNx: {rNx}")
        print(f"eTL: {eTL}")
        print(f"eTC: {eTC}")
        print(f"firstacq: {firstacq}")
        print(f"navgs: {navgs}")

    # Initialize the kspace data array
    kspace = np.zeros((navgs, nslices, ncoils, rNx, eNy + 1), dtype=np.complex64)

    # Loop through the rest of the acquisitions and fill the data array with the kspace data
    with tqdm(total=dset.number_of_acquisitions() - firstacq) as pbar:
        for acqnum in range(firstacq, dset.number_of_acquisitions()):
            acq    = dset.read_acquisition(acqnum)
            slice1 = acq.idx.slice
            y      = acq.idx.kspace_encode_step_1
            avg    = acq._head.idx.average

            # each acquisition is a 2D array of shape (coil, rNx) complex
            kspace[avg, slice1, :, :, y] = acq.data
            pbar.update(1)

    # reorder the data from even and odd number of slices being interleaved to all even slices first and then all odd slices
    kspace = reorder_kspace_slices(kspace)

    if verbose:
        print(f"build_np_from_ismrmrd took {time.time() - start} seconds")
    
    return kspace


def get_patient_id_rumc(fpath):
    '''
    Get the patient id from the file path.
    Assumes the file path has the format: /mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_rumc/pst_example/raw/<patient_id>/<date>/<file_name>.mrd
    '''
    # Convert fpath to a string, then split it using the appropriate separator
    fpath_str = str(fpath)
    patient_id = fpath_str.split("/")[-3]
    return patient_id


def create_h5_from_mrd_rumc(fpath: str, workdir: str, verbose=False, debug=False) -> None:
    '''
    '''

    print(f"Processing patient T2W MRD file with path: {fpath}")

    # get the patient id from the file path
    patient_id = get_patient_id_rumc(fpath)
    print(f"\tPatient ID RUMC: {patient_id}")

    # create the h5 file path
    fpath_hf = os.path.join(workdir, "output", "h5s", f"{patient_id}_pst_T2.h5")

    # check if the h5 file already exists
    if not os.path.exists(fpath_hf):
        with h5py.File(fpath_hf, 'w') as hf:
            print(f"created h5 file at {fpath_hf}")
    else:
        print(f"\tH5 file already exists: {fpath_hf}")

    # Verify and create missing components in the H5 file.
    with h5py.File(fpath_hf, 'r+') as hf:

        if not has_kspace_key(fpath_hf):
            kspace = build_np_from_ismrmrd_rumc(fpath, DEBUG=False, verbose=True)
            hf.create_dataset('kspace', data=kspace)
            print(f"\tcreated kspace key and filled with kspace data in h5 file: {fpath_hf}")

        if not has_correct_shape(fpath_hf):
            raise Exception(f"kspace shape is not correct. Shape: {hf['kspace'].shape}")
        
        # if the headers key is not present in the h5 file yet 
        # then add the headers key and fill it with headers data from the .mrd file
        # if not has_headers_key(fpath_hf):
        #     rumc_headers_mrd  = get_headers_from_ismrmrd_rumc(fpath, verbose=True)
        #     rumcheaders_dict = convert_ismrmrd_headers_to_dict(rumc_headers_mrd)
        #     header_bytes      = encode_umcg_header_to_bytes(umcg_to_nyu_dict=rumc_headers_dict)
        #     hf.create_dataset('ismrmrd_header', data=header_bytes)


# set workdir based on wsl execution RUMC
if os.path.exists("/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_rumc/workspace"):
    WORKDIR_RUMC = "/mnt/c/Users/qvloh/Documents/phd_lok/datasets/prostate_rumc/workspace"
else: 
    WORKDIR_RUMC = "C:\\Users\\qvloh\\Documents\\phd_lok\\datasets\\prostate_rumc\\workspace"

print(f"workdir rumc: {WORKDIR_RUMC}")

create_h5_from_mrd_rumc(FPATH_RUMC_MRD_SAMPLE, WORKDIR_RUMC, verbose=True)

# Load the Kspaces (UMCG, NYU, or RUMC) into RAM

In [ ]:
ksp_umcg = h5py.File(FPATH_UMCG_H5_SAMPLE, 'r')["kspace"][()]

In [ ]:
ksp_nyu = h5py.File(FPATH_NYU_H5_SAMPLE, 'r')["kspace"][()]

In [ ]:
ksp_rumc = h5py.File(FPATH_RUMC_H5_SAMPLE, 'r')["kspace"][()]

# STEP 1 - Remove the zero-padding from the data

In [ ]:
if 'ksp_umcg' in globals():
    print("UMCG")
    ksp_umcg = remove_zero_padding(ksp_umcg, verbose=True)

if 'ksp_nyu' in globals():
    print("\nNYU")
    ksp_nyu  = remove_zero_padding(ksp_nyu,  verbose=True)

if 'ksp_rumc' in globals():
    print("\nRUMC")
    ksp_rumc = remove_zero_padding(ksp_rumc, verbose=True)

# STEP 2 - Remove 50 lines left and 50 lines right of the UMCG kspace so that the dimensions are the same as NYU.
Look at the UMCG kspace zero padding removed. We will crop in k-space because there seems to be a similar field of view for both NYU and UMCG. So we will go crop kspace for UMCG to 640x451. 


In [ ]:
ksp_umcg, new_hdrs = crop_kspace_in_phase_direction(ksp_umcg, target_shape=(3, 31, 20, 640, 451), fpath_mrd=FPATH_UMCG_MRD_SAMPLE, verbose=True)
print(f"Cropped kspace shape is now: {ksp_umcg.shape}")

# STEP 3 - Normalize the UMCG data to the same range as NYU

In [ ]:
# Based on one patient of the NYU dataset the max_magnitude reference: 
max_magnitude_reference = 0.006096669    # comes from. but then i do not have to load in the reference kspace
ksp_umcg = normalize_to_reference(ksp_umcg, max_magnitude_reference)

# STEP 4 - Write to h5 together with the ismrmrd header. Add the changed header to the new h5 file, othwerise the RIM doesn't understand the phase dimension anymore.


In [ ]:
# get the patient id for the file name
with h5py.File(FPATH_UMCG_H5_SAMPLE, 'r') as f:
    pat_id = f.attrs["patient_id"]

# create the h5 file path
fpath_fixed_ksp_umcg = Path(WORKDIR, "tmp", f"{pat_id}_pst_T2W_transformed_v3.h5")

# create the h5 file if it does not exist
if not os.path.exists(fpath_fixed_ksp_umcg):
    with h5py.File(fpath_fixed_ksp_umcg, 'w') as hf:
        print(f"created h5 file at {fpath_fixed_ksp_umcg}")
        hf.create_dataset('kspace', data=ksp_umcg)
        hf.create_dataset('ismrmrd_header', data=new_hdrs)

        hf.attrs['acquisition'] = 'AXT2'
        hf.attrs['max'] = 0.00054
        hf.attrs['norm'] = 0.149
        hf.attrs['patient_id'] = pat_id
        hf.attrs['patient_id2'] = ""

# Compare the UMCG and NYU prostate kspaces

## Shape, Dtype, Zero-Padding

In [ ]:
# UMCG
if 'ksp_umcg' in globals():
    zp_umcg, zp_idxs_umcg = calculate_zero_padding_PE(ksp_umcg)
    print("UMCG - kspace shape: ", ksp_umcg.shape)
    print("UMCG - kspace dtype: ", ksp_umcg.dtype)
    print(f"UMCG - Amount of zero padding: {zp_umcg} in phase encoding direction.")
    print(f"UMCG - Data is acquired until {ksp_umcg.shape[-1] - zp_umcg}")
    print()

# NYU
if 'ksp_nyu' in globals():
    zp_nyu, zp_idxs_nyu = calculate_zero_padding_PE(ksp_nyu)
    print("NYU  - kspace shape: ", ksp_nyu.shape)
    print("NYU  - kspace dtype: ", ksp_nyu.dtype)
    print(f"NYU  - Amount of zero padding: {zp_nyu} in phase encoding direction.")
    print(f"NYU  - Data is acquired until {ksp_nyu.shape[-1] - zp_nyu}")
    print()

# RUMC
if 'ksp_rumc' in globals():
    zp_rumc, zp_idxs_rumc = calculate_zero_padding_PE(ksp_rumc)
    print("RUMC - kspace shape: ", ksp_rumc.shape)
    print("RUMC - kspace dtype: ", ksp_rumc.dtype)
    print(f"RUMC - Amount of zero padding: {zp_rumc} in phase encoding direction.")
    print(f"RUMC - Data is acquired until {ksp_rumc.shape[-1] - zp_rumc}")

# Kspace Averages Visualization

In [ ]:
# UMCG
if True:
    print("UMCG")
    ft = f'\n\n\n\n\nThere is a chunck of zero-padding in the phase direction.'
    visualize_kspace_averages(ksp_umcg, slice_idx=0, coil_idx=0, outdir=TEMPDIR, fname_id="UMCG", figtext=ft)

# NYU
if True:
    print("\nNYU")
    ft = f'\n\n\n\n\nThere seems to be no zero padding.'
    visualize_kspace_averages(ksp_nyu, slice_idx=0, coil_idx=0, outdir=TEMPDIR, fname_id="NYU", figtext=ft)

# RUMC
if False:
    print("\nRUMC")
    # ft = f'\n\n\n\n\nThere seems to be no zero padding.'
    ft = "\n\n\n\n\nThere seems to be no zero padding."
    ft += "\n\n\n\n\nThere seems to be only 2 averages."
    visualize_kspace_averages(ksp_rumc, slice_idx=0, coil_idx=0, outdir=TEMPDIR, fname_id="RUMC", figtext=ft)

# VIEW - UMCG headers in MRD format & NYU in ISMRMRD format (a Comparison)


In [ ]:
print("UMCG")
umcg_headers = get_headers_from_ismrmrd(FPATH_UMCG_MRD_SAMPLE, verbose=True)

print("\n\n\n\n\nNYU")
nyu_headers  = get_headers_from_h5(FPATH_NYU_H5_SAMPLE, verbose=False)

if False:
    print("\n\n\n\n\nRUMC")
    rumc_headers = get_headers_from_h5(FPATH_RUMC_H5_SAMPLE, verbose=False)

# Verify that both H5 files - UMCG and NYU - have the same header format


In [ ]:
umcg_headers = get_headers_from_h5(FPATH_UMCG_H5_SAMPLE, verbose=True)

In [ ]:
nyu_headers  = get_headers_from_h5(FPATH_NYU_H5_SAMPLE, verbose=True)

# RSS Reconstructions for UMCG, NYU and RUMC

In [ ]:
if 'ksp_umcg' in globals():
    print("UMCG")
    umcg_rss = rss_recon_from_ksp(ksp_umcg, averages=(0,1), verbose=True, printkey="UMCG")
    save_numpy_rss_as_nifti(umcg_rss, "umcg_rss", dir=TEMPDIR)

if 'ksp_nyu' in globals():
    print("\nNYU")
    nyu_rss = rss_recon_from_ksp(ksp_nyu,  averages=(0,1), verbose=True, printkey="NYU")
    save_numpy_rss_as_nifti(nyu_rss, "nyu_rss", dir=TEMPDIR)

if 'ksp_rumc' in globals():
    print("\nRUMC")
    rumc_rss = rss_recon_from_ksp(ksp_rumc,  averages=(0,1), verbose=True, printkey="RUMC")
    save_numpy_rss_as_nifti(rumc_rss, "rumc_rss", dir=TEMPDIR)

# What are the MAX and NORM of the RSS reconstruction and what are these values in the h5 file?

In [ ]:
print(nyu_rss.shape)

norm = np.linalg.norm(nyu_rss[:])
max = np.max(nyu_rss[:])

print(f"max and norm of nyu_rss: {max} and {norm}")

with h5py.File(FPATH_NYU_H5_SAMPLE, 'r') as f:
    print(f.attrs.keys())
    # print the max and norm of the h5 file
    print(f"max and norm of nyu_rss: {f.attrs['max']} and {f.attrs['norm']}")


# Use center_crop_im(im_3d: np.ndarray, crop_to_size: Tuple[int, int]) -> np.ndarray: to do the cropping
nyu_rss_crop = center_crop_im(nyu_rss, (320, 320))
# calculate the norm and max of the cropped image
norm = np.linalg.norm(nyu_rss_crop[:])
max = np.max(nyu_rss_crop[:])
print(f"max and norm of nyu_rss_crop: {max} and {norm}")


# Visualize each institutes k-spaces

In [ ]:
if 'ksp_umcg' in globals():
    print("UMCG ")
    safe_kspace_slice_to_png(ksp_umcg, TEMPDIR, titlepart="UMCG", do_log=True, slice_no=0, coil_no=0, avg_to_add=(0,1))

if 'ksp_nyu' in globals():
    print("\nNYU ")
    safe_kspace_slice_to_png(ksp_nyu,  TEMPDIR, titlepart="NYU",  do_log=True, slice_no=0, coil_no=0, avg_to_add=(0,1))

if 'ksp_rumc' in globals():
    print("\nRUMC ")
    safe_kspace_slice_to_png(ksp_rumc, TEMPDIR, titlepart="RUMC", do_log=True, slice_no=0, coil_no=0, avg_to_add=(0,1))

# Create new visualizations of the zero-pad removed kspaces

In [ ]:
if 'ksp_umcg' in globals():
    print("UMCG")
    safe_kspace_slice_to_png(ksp_umcg, TEMPDIR, titlepart="UMCG_nzp", do_log=True, slice_no=0, coil_no=0, avg_to_add=(0,1))
    umcg_rss_nzp = rss_recon_from_ksp(ksp_umcg, averages=(0,1), verbose=True, printkey="UMCG")
    save_numpy_rss_as_nifti(umcg_rss_nzp, "umcg_rss_nzp", dir=TEMPDIR)

if 'ksp_nyu' in globals():
    print("\nNYU")
    safe_kspace_slice_to_png(ksp_nyu,  TEMPDIR, titlepart="NYU_nzp",  do_log=True, slice_no=0, coil_no=0, avg_to_add=(0,1))
    nyu_rss_nzp  = rss_recon_from_ksp(ksp_nyu,  averages=(0,1), verbose=True, printkey="NYU")
    save_numpy_rss_as_nifti(nyu_rss_nzp, "nyu_rss_nzp", dir=TEMPDIR)

if 'ksp_rumc' in globals():
    print("\nRUMC")
    safe_kspace_slice_to_png(ksp_rumc, TEMPDIR, titlepart="RUMC_nzp", do_log=True, slice_no=0, coil_no=0, avg_to_add=(0,1))
    rumc_rss_nzp = rss_recon_from_ksp(ksp_rumc, averages=(0,1), verbose=True, printkey="RUMC")
    save_numpy_rss_as_nifti(rumc_rss_nzp, "rumc_rss_nzp", dir=TEMPDIR)


# Check if the UMCG attributes exist correctly

In [ ]:
with h5py.File(FPATH_UMCG_H5_SAMPLE, 'r') as f:
    print(f"keys in h5: {list(f.keys())}")
    attrs = dict(f.attrs)
    print("attributes length = ", len(attrs))
    for k, v in attrs.items():
        print(f"\t{k}: {v}")

# get only the patient id
with h5py.File(FPATH_UMCG_H5_SAMPLE, 'r') as f:
    # get the patient_id attribute of the h5 file
    pat_id = f.attrs["patient_id"]
    print(pat_id)

# Add the zero padding to the right side for a nice looking anatomy. NYU


In [ ]:
# Temporary pad the NYU kspace to 640 phase and also the UMCG
ksp_nyu_padded = np.pad(ksp_nyu, ((0,0), (0,0), (0,0), (0,0), (0, 640-ksp_nyu.shape[-1])), mode='constant')
print(f"ksp_nyu_padded.shape = {ksp_nyu_padded.shape}")

Do RSS and safe as nifti

In [ ]:
# calculate the RSS of the zero-padded kspace and save it as a nifti
nyu_nzp_reduced_rss = rss_recon_from_ksp(ksp_nyu_padded, averages=(0,1), verbose=True, printkey="nyu")
save_numpy_rss_as_nifti(nyu_nzp_reduced_rss, "nyu_padded_for_img_space", dir=TEMPDIR)
del ksp_nyu_padded, nyu_nzp_reduced_rss

# Create Nifti of the UMCG DATA (RSS Recon)

In [ ]:
# STEP 3: calculate the RSS of the reduced kspace and save it as a nifti
umcg_nzp_reduced_rss = rss_recon_from_ksp(ksp_umcg, averages=(0,1), verbose=True, printkey="UMCG")
save_numpy_rss_as_nifti(umcg_nzp_reduced_rss, "umcg_init_nzp_rss", dir=TEMPDIR)

# STEP 4: Pad the UMCG kspace to 640 phase
ksp_umcg_padded = np.pad(ksp_umcg, ((0,0), (0,0), (0,0), (0,0), (0, 640-nyu_shape[-1])), mode='constant')
print(f"\nSTEP 4: ksp_umcg_padded.shape = {ksp_umcg_padded.shape}")

# STEP 5: calculate the RSS of the reduced kspace and save it as a nifti
umcg_nzp_reduced_rss = rss_recon_from_ksp(ksp_umcg_padded, averages=(0,1), verbose=True, printkey="UMCG")
save_numpy_rss_as_nifti(umcg_nzp_reduced_rss, "umcg_padded_for_img_space", dir=TEMPDIR)

del ksp_umcg_padded, umcg_nzp_reduced_rss

# Analyize both the NYU and UMCG kspace and check for differences

In [ ]:
analyze_kspace_3d_vol(ksp_umcg, "K-Space UMCG")
hist_real_and_imag(ksp_umcg, "K-Space UMCG")

analyze_kspace_3d_vol(ksp_nyu, "K-Space NYU")
hist_real_and_imag(ksp_nyu, "K-Space NYU")

# After Normalization check that UMCG has similar ranges to NYU data.

In [ ]:
print(f"UMCG")
analyze_kspace_3d_vol(ksp_umcg)

print(f"\nNYU")
analyze_kspace_3d_vol(ksp_nyu)

# The UMCG kspace now has correct dimensions it seems and also the correct real and imaginary ranges of values, so write to h5 together with the ismrmrd header. The header needs to be changed. Otherwise the RIM model doesnt understand the some things about the k-space


In [ ]:
def get_headers_from_mrd_wrapper(fpath_mrd: str) -> bytes:
    """
    Wrapper function to get the headers from a .mrd file and transform them to the NYU format.
    """

    # get the headers from the .mrd file
    umcg_headers_mrd  = get_headers_from_ismrmrd(fpath_mrd, verbose=False)

    # transform the headers a dictionary
    umcg_headers_dict = convert_ismrmrd_headers_to_dict(umcg_headers_mrd)

    # Set the correct matrix size and encoding limits for the NYU data
    t = "{http://www.ismrm.org/ISMRMRD}"
    umcg_headers_dict[f"{t}ismrmrdHeader"][f"{t}encoding"][f"{t}encodedSpace"][f"{t}matrixSize"][f"{t}y"] = str(450)
    umcg_headers_dict[f"{t}ismrmrdHeader"][f"{t}encoding"][f"{t}encodingLimits"][f"{t}kspace_encoding_step_1"][f"{t}maximum"] = str(450)
    umcg_headers_dict[f"{t}ismrmrdHeader"][f"{t}encoding"][f"{t}encodingLimits"][f"{t}kspace_encoding_step_1"][f"{t}center"] = str(225)

    header_bytes = encode_umcg_header_to_bytes(umcg_to_nyu_dict=umcg_headers_dict)
    return header_bytes

# get the patient id.
with h5py.File(FPATH_UMCG_H5_SAMPLE, 'r') as f:
    pat_id = f.attrs["patient_id"]

# create the h5 file path
fpath_fixed_ksp_umcg = Path(WORKDIR, "tmp", f"{pat_id}_pst_T2W_transformed_v2.h5")
transformed_hdrs = get_headers_from_mrd_wrapper(FPATH_UMCG_MRD_SAMPLE)

# create the h5 file if it does not exist
if not os.path.exists(fpath_fixed_ksp_umcg):
    with h5py.File(fpath_fixed_ksp_umcg, 'w') as hf:
        print(f"created h5 file at {fpath_fixed_ksp_umcg}")
        hf.create_dataset('kspace', data=ksp_umcg)
        hf.create_dataset('ismrmrd_header', data=transformed_hdrs)

        hf.attrs['acquisition'] = 'AXT2'
        hf.attrs['max'] = 0.00054
        hf.attrs['norm'] = 0.149
        hf.attrs['patient_id'] = pat_id
        hf.attrs['patient_id2'] = ""

# Analyse the Actual Differences between the T2W dicom and the RSS Deep Learning Reconstruction.

In [ ]:
def load_nifti(file_path, as_numpy=False, verbatim=False):
    """
    Load a NIfTI file using SimpleITK.
    
    Parameters:
        file_path (str): Path to the NIfTI file.
        as_numpy (bool): Whether to return the image as a numpy array.
        verbatim (bool): Whether to print details about the loaded image/array.
        
    Returns:
        SimpleITK.Image or numpy.ndarray: The loaded image.
    """
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} not found.")
    
    sitk_img = sitk.ReadImage(file_path)
    
    if as_numpy:
        array = sitk.GetArrayFromImage(sitk_img)

        if verbatim:
            print(f"\tshape: {array.shape}")
            print(f"\tmean:  {np.mean(array)}")
            print(f"\tstd:   {np.std(array)}")
            print(f"\tmin:   {np.min(array)}")
            print(f"\tmax:   {np.max(array)}")
        return array

    else:
        if verbatim:
            print(f"\tDimensions: {sitk_img.GetDimension()}")
            print(f"\tSize:       {sitk_img.GetSize()}")
            print(f"\tSpacing:    {sitk_img.GetSpacing()}")
            print(f"\tOrigin:     {sitk_img.GetOrigin()}")
            print(f"\tDirection:  {sitk_img.GetDirection()}")
            print(f"\tPixel Type: {sitk_img.GetPixelIDTypeAsString()}")

    return sitk_img

# Load the DICOM array of the correct patient
print(pat_id)

nifti_fpath = Path(WORKDIR, 'input', 'sectra_export_pat0001_pat0010_niftis', pat_id, '2022-04-07', 'tse2d1_25_1.3.12.2.1107.5.2.19.46133.2022040713122344044291325.0.0.0.nii.gz')
print(nifti_fpath)

# load the DICOM array of the same patient
dcm_arr = load_nifti(nifti_fpath, as_numpy=True, verbatim=True)

plt.imshow(dcm_arr[15, :, :], cmap='gray')
plt.show()

# load the RSS DL array of the same patient

# Perform analysis on the differences between the two.

# What differences are observed.

# Small test to create a zero-padding function around each edge.

In [ ]:
def zero_pad_kspace(ksp: np.ndarray, desired_shape: tuple, verbose=False) -> np.ndarray:
    '''
    Arguments:
        - ksp: 2D numpy array of shape (readout, phase) complex
        - desired_shape: tuple of desired shape (readout, phase)
    Returns:
        - padded_ksp: 2D numpy array of shape (readout, phase) complex
    '''
    diff = np.array(desired_shape) - np.array(ksp.shape)
    pad_left  = diff // 2
    pad_right = diff - pad_left
    
    padded_ksp = np.pad(ksp, ((pad_left[0], pad_right[0]), (pad_left[1], pad_right[1])), mode='constant')
    
    if verbose:
        print(f"current shape:    {ksp.shape}")
        print(f"desired shape:    {desired_shape}")
        print(f"padded_ksp shape: {padded_ksp.shape}")

    return padded_ksp


ksp = np.random.randn(640, 451) + 1j * np.random.randn(640, 451)
ksp_zp = zero_pad_kspace(ksp, (1200, 1200), verbose=True)

# visualize the original kspace and the zero padded kspace seperately
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.abs(ksp), cmap='gray')
axs[0].set_title("Original k-space")
axs[1].imshow(np.abs(ksp_zp), cmap='gray')
axs[1].set_title("Zero padded k-space")
plt.show()

# Let's take a look at the created h5 file from MRD

In [ ]:
temp_h5_file = Path(WORKDIR, "output", "h5s", "ANON2784451_pst_T2.h5")

with h5py.File(temp_h5_file, 'r') as f:
    # print all keys
    print(f"keys in h5: {list(f.keys())}")
    print(f"kspace shape = {f['kspace'].shape}")
    print(f"kspace dtype = {f['kspace'].dtype}")

    # print all the attributes of the h5 file
    attrs = dict(f.attrs)
    print("attributes length = ", len(attrs))
    for k, v in attrs.items():
        print(f"\t{k}: {v}")

# Look at NYU RSS reconstruction.
There is an RSS reconstruction in the h5 file. What is it's 'max' and 'norm'?

In [ ]:
with h5py.File(FPATH_NYU_H5_SAMPLE, 'r') as hf:
    print(f"keys in h5: {list(hf.keys())}")

    # load in the reconstruction called reconstruction_rss
    nyu_rss = hf["reconstruction_rss"][()]
    print(f"nyu_rss.shape = {nyu_rss.shape}")
    max = np.max(nyu_rss[:])
    norm = np.linalg.norm(nyu_rss[:])
    print(f"max = {max}")
    print(f"norm = {norm}")

    # print all attributes of the h5 file
    attrs = dict(hf.attrs)
    for k, v in attrs.items():
        print(f"\t{k}: {v}")

    # pick a random slice index from the first dimension of the RSS reconstruction
    sliceee = np.random.randint(0, nyu_rss.shape[0])
    plt.imshow(np.abs(nyu_rss[sliceee, :, :]), cmap='gray')
    plt.title(f"Slice {sliceee} of the RSS reconstruction")
    plt.show()

# Read in the calibration data of the same patient - Try to convert it into a numpy array with a similar shape to the acquisition. (30, 20, 640, 32) for example

In [ ]:
# create a function that recursively prints all keys and values in enc.
def print_all_keys_and_values(enc, level=0):
    for k, v in enc.items():
        print(f"{' ' * level}{k}: {v}")
        if isinstance(v, dict):
            print_all_keys_and_values(v, level=level+1)


def build_calib_array_from_ismrmrd_umcg(fpath: str, DEBUG=False, verbose=True):

    # get_headers_from_ismrmrd(fpath, verbose=True)
    # return None

    # Read the header and get encoding information
    dset = ismrmrd.Dataset(fpath, create_if_needed=False)
    header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
    enc = header.encoding[0]

    # Determine some parameters of the acquisition
    ncoils     = header.acquisitionSystemInformation.receiverChannels
    nslices    = enc.encodingLimits.kspace_encoding_step_1.maximum
    firstacq   = 1 if DEBUG else get_first_acquisition(dset)

    encspace_mat_x = enc.encodedSpace.matrixSize.x
    encspace_mat_y = enc.encodedSpace.matrixSize.y
    encspace_mat_z = enc.encodedSpace.matrixSize.z
    recspace_mat_y = enc.reconSpace.matrixSize.y
    recspace_mat_x = enc.reconSpace.matrixSize.x
    recspace_mat_z = enc.reconSpace.matrixSize.z

    print(f"enc space x,y,z = {encspace_mat_x}, {encspace_mat_y}, {encspace_mat_z}")
    print(f"rec space x,y,z = {recspace_mat_x}, {recspace_mat_y}, {recspace_mat_z}")
    print(f"encspace_mat_y = {encspace_mat_y}")
    print(f"encspace_mat_x = {encspace_mat_x}")
    print(f"ncoils = {ncoils}")
    print(f"nslices = {nslices}")

    # Initialize the kspace data array
    calib = np.zeros((nslices, ncoils, encspace_mat_x, encspace_mat_y), dtype=np.complex64)
    print(f"Current initialized calibration shape = {calib.shape}")

    # Loop through the rest of the acquisitions and fill the data array with the kspace data
    for acqnum in range(firstacq, dset.number_of_acquisitions()):
        acq    = dset.read_acquisition(acqnum)
        slice1 = acq.idx.slice
        y      = acq.idx.kspace_encode_step_1

        print(f"acq num = {acqnum} with shape {acq.data.shape}, with slice1 = {slice1} and y = {y}")

        # each acquisition is a 2D array of shape (coil, rNx) complex
        # kspace[slice1, :, :, y] = acq.data

    # reorder the data from even and odd number of slices being interleaved to all even slices first and then all odd slices
    # kspace = reorder_kspace_slices(kspace)
    
    return kspace


kspace = build_calib_array_from_ismrmrd_umcg(FPATH_UMCG_MRD_SAMPLE_CALIB)


# SANDBOX TEMP

In [ ]:

def build_np_from_ismrmrd_umcg(fpath: str, DEBUG=False, verbose=True):
    '''
    Arguments:
        - fpath: path to the .mrd file
    
    Returns:
        - kspace: numpy array of kspace data in shape (navgs, nslices, ncoils, rNx, eNy + 1) complex
    '''
    if verbose:
        print(f"Building kspace from .mrd file")

    # lets time this whole function and print it
    start = time.time()

    # Read the header and get encoding information
    dset = ismrmrd.Dataset(fpath, create_if_needed=False)
    header = ismrmrd.xsd.CreateFromDocument(dset.read_xml_header())
    enc = header.encoding[0]

    # Determine some parameters of the acquisition
    ncoils     = header.acquisitionSystemInformation.receiverChannels
    nslices    = enc.encodingLimits.slice.maximum + 1 if enc.encodingLimits.slice is not None else 1
    eNy        = enc.encodedSpace.matrixSize.y
    rNx        = enc.reconSpace.matrixSize.x
    # eTL        = 25 if DEBUG else echo_train_length(dset)
    # eTC        = 11 if DEBUG else echo_train_count(dset, echo_train_len=eTL)
    firstacq   = 1 if DEBUG else get_first_acquisition(dset)
    navgs      = 3 if DEBUG else get_num_averages(firstacq=firstacq, dset=dset)

    print(f"ncoils = {ncoils}")
    print(f"nslices = {nslices}")
    print(f"eNy = {eNy}")
    print(f"rNx = {rNx}")
    # print(f"eTL = {eTL}")
    # print(f"eTC = {eTC}")
    print(f"firstacq = {firstacq}")
    print(f"navgs = {navgs}")

    # # Initialize the kspace data array
    # kspace = np.zeros((navgs, nslices, ncoils, rNx, eNy + 1), dtype=np.complex64)

    # # Loop through the rest of the acquisitions and fill the data array with the kspace data
    # with tqdm(total=dset.number_of_acquisitions() - firstacq) as pbar:
    #     for acqnum in range(firstacq, dset.number_of_acquisitions()):
    #         acq    = dset.read_acquisition(acqnum)
    #         slice1 = acq.idx.slice
    #         y      = acq.idx.kspace_encode_step_1
    #         avg    = acq._head.idx.average

    #         # each acquisition is a 2D array of shape (coil, rNx) complex
    #         kspace[avg, slice1, :, :, y] = acq.data
    #         pbar.update(1)

    # # reorder the data from even and odd number of slices being interleaved to all even slices first and then all odd slices
    # kspace = reorder_kspace_slices(kspace)

    # if verbose:
    #     print(f"build_np_from_ismrmrd took {time.time() - start} seconds")
    
    # return kspace
    return None
    

kspace = build_np_from_ismrmrd_umcg(FPATH_UMCG_MRD_SAMPLE, DEBUG=True, verbose=True)

# Explore the NYU calibration data in more detail

In [ ]:
with h5py.File(FPATH_NYU_H5_SAMPLE, 'r') as hf:
    print(f"keys in h5: {list(hf.keys())}")

    # print calibration shape
    calib_shape = hf["calibration_data"][()].shape
    print(f"calibration shape = {calib_shape}")

    # kspace_shape = hf["kspace"][()].shape
    # print(f"kspace shape = {kspace_shape}")
    nyu_calib = hf["calibration_data"][()]

    # visualize the first slice 1 coil of the calibration data
    print(nyu_calib.dtype)

    calib_slice = nyu_calib[0, 0, ...]
    print(calib_slice.shape)

    # visualize this slice in grey scale with the absolute
    plt.imshow(np.abs(calib_slice), cmap='gray')
    plt.title("Calibration slice 0 coil 0")
    plt.show()

    # Visualize the calib_slice with ifft
    plt.imshow(np.abs(np.fft.ifftshift(np.fft.ifft2(np.fft.ifftshift(calib_slice)))), cmap='gray')
    # save this on to tempdir
    plt.savefig(Path(TEMPDIR, "calib_slice_0_coil_0.png"))

    # visualize without all the shifts
    plt.imshow(np.abs(np.fft.ifft2(calib_slice)), cmap='gray')


# Aggregate all the T2 TSE files into a list and tar them together for transfer to Habrok

In [ ]:
def get_all_t2_tra_paths_to_transfer_to_habrok(target_dir: Path, verbose=False) -> List[Path]:
    pattern1 = str(Path(target_dir, "*", "meas_*_T2_TSE_tra_obl-out*.mrd"))
    pattern2 = str(Path(target_dir, "*", "meas_*_t2_tse_traobl*.mrd"))

    t2_tra_paths = case_insensitive_glob(pattern1)
    t2_tra_paths += case_insensitive_glob(pattern2)

    if verbose:
        print(f"Found {len(t2_tra_paths)} T2 tra paths")
        print("\tT2 path: ", t2_tra_paths)

    return t2_tra_paths

target_dir = Path(WORKDIR, 'output', 'anon_kspaces')
transfer_paths = get_all_t2_tra_paths_to_transfer_to_habrok(target_dir, verbose=True)

fpath_tar = Path(WORKDIR, 'output', 'anon_kspaces_t2_tse_tra_1_2.tar')
with tarfile.open(fpath_tar, "w") as tar:
    for fpath in transfer_paths:
        relative_path = os.path.relpath(fpath, start=target_dir)
        tar.add(fpath, arcname=relative_path)